# 클러스터링

- 화합물에 대한 클러스터링 수행 예
- Fingerprint를 이용한 Scaffold 기반 클러스터링 수행
- 데이터 처리 모듈 [datamol](https://datamol.io/)에서 다양한 데이터 전처리 함수와 클러스터링 함수 제공

# Import

In [ ]:
!pip install rdkit datamol tqdm mols2grid

In [ ]:
import pandas as pd
import datamol as dm
from tqdm import tqdm
import mols2grid

%config InlineBackend.figure_format = 'retina'
pd.options.display.float_format = '{:,.3f}'.format # 소수점 표시 지정

# 데이터

- hERG 저해제 데이터
- The human ether-a-go-go-related gene (hERG)
 - 심장에서 칼륨(K+)이온의 흐름을 조절해 심장박동을 조정하는 유전자
 - 심장 독성 발생과 관련된 유전자

In [ ]:
url = "https://raw.githubusercontent.com/PatWalters/yamc/main/data/HERG.smi"
df = pd.read_csv(url,sep=" ",names=["SMILES","Name","pIC50"])
print(df.shape)
df[:3]

(4042, 3)


,SMILES,Name,pIC50
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.025
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.328
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.072


### mol 객체 얻기

- from_df 함수 사용


In [ ]:
df['mol'] = dm.from_df(df, smiles_column="SMILES")
df.head()

,SMILES,Name,pIC50,mol
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.025,<rdkit.Chem.rdchem.Mol object at 0x7feb88aceba0>
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.328,<rdkit.Chem.rdchem.Mol object at 0x7feb88ace9e0>
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.072,<rdkit.Chem.rdchem.Mol object at 0x7feb88acea50>
3,CC(C)C(CCCN(C)CCc1ccc(c(c1)OC)OC)(C#N)c2ccc(c(...,CHEMBL6966,6.831,<rdkit.Chem.rdchem.Mol object at 0x7feb88acec10>
4,CCOC(=O)N1CCC(=C2c3ccc(cc3CCc4c2nccc4)Cl)CC1,CHEMBL998,6.429,<rdkit.Chem.rdchem.Mol object at 0x7feb88acec80>


### 데이터 처리 함수 정의

- 분자의 가장 큰 링의 크기를 얻는 함수를 정의

In [ ]:
def max_ring_size(mol):
    ri = mol.GetRingInfo()
    atom_rings = ri.AtomRings()
    if len(atom_rings) == 0:
        return 0
    else:
        return max([len(x) for x in ri.AtomRings()])

## 분자의 특성 추가

- 추가할 특성을 얻는 함수를 딕셔너리로 만든다
- 위에서 정의한 max_ring_size 함수도 추가

In [ ]:
my_prop_dict = {
    "mw" : dm.descriptors.mw,
    "logp" : dm.descriptors.clogp,
    "hbd" : dm.descriptors.n_lipinski_hbd,
    "hba" : dm.descriptors.n_lipinski_hba,
    "max_ring_size" : max_ring_size
}

- datamol이 제공하는 **batch_compute_many_descriptors**를 사용하면 해당 특성들을 한번에 추가한다
 - add_properties를 True로 설정하면 모든 지원되는 descriptors를 계산한다
 - progress=True 진행 프로그레스를 보여준다

In [ ]:
prop_df = dm.descriptors.batch_compute_many_descriptors(df.mol,properties_fn=my_prop_dict,
                add_properties=False, progress=True)

  0%|          | 0/4042 [00:00<?, ?it/s]

In [ ]:
prop_df[:3]

,mw,logp,hbd,hba,max_ring_size
0,440.178,4.628,1,5,6
1,379.170,3.678,1,5,6
2,324.147,2.519,1,5,6


- 원본 데이터프레임과 합친다 (컬럼 방향으로)

In [ ]:
df = pd.concat([df,prop_df],axis=1)
print(df.shape)
df[:3]

(4042, 9)


,SMILES,Name,pIC50,mol,mw,logp,hbd,hba,max_ring_size
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.025,<rdkit.Chem.rdchem.Mol object at 0x7feb88aceba0>,440.178,4.628,1,5,6
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.328,<rdkit.Chem.rdchem.Mol object at 0x7feb88ace9e0>,379.170,3.678,1,5,6
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.072,<rdkit.Chem.rdchem.Mol object at 0x7feb88acea50>,324.147,2.519,1,5,6


# Lipinski Rule of 5 (Ro5)

- 통상 아래과 같이 처리한다
```python
df = df[df['mw'] <= 500]
df = df[df['logp'] <= 5]
df = df[df['hbd'] <= 5]
df = df[df['hba'] <= 10]
```

- 여기서는 위와 같은 조건을 모두 만족하는 불리언 리스트를 만들어 적용하겠다

In [ ]:
ro5_ok = (df.mw <= 500) & (df.logp <=5) & (df.hbd <= 5) & (df.hba <= 10)
print(ro5_ok.sum())
ro5_ok[:10]

3476


0     True
1     True
2     True
3    False
4     True
5     True
6     True
7     True
8    False
9     True
dtype: bool

## 데이터 필터링

In [ ]:
print(df.shape)
df = df[ro5_ok] # Ro5 조건을 만족하는 분자만 선택한 결과
print(df.shape)
df[:3]

(4042, 9)
(3476, 9)


,SMILES,Name,pIC50,mol,mw,logp,hbd,hba,max_ring_size
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.025,<rdkit.Chem.rdchem.Mol object at 0x7feb88aceba0>,440.178,4.628,1,5,6
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.328,<rdkit.Chem.rdchem.Mol object at 0x7feb88ace9e0>,379.170,3.678,1,5,6
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.072,<rdkit.Chem.rdchem.Mol object at 0x7feb88acea50>,324.147,2.519,1,5,6


# 클러스터링

- datamol이 제공하는 [cluster_mols](https://docs.datamol.io/stable/api/datamol.cluster.html#datamol.cluster.cluster_mols) 함수를 사용
- 기본적으로 Morgan fingerprints 기반의 [Butina](https://pubs.acs.org/doi/pdf/10.1021/ci9803381) 알고리즘 사용하며, threshold를 지정할 수 있다

In [ ]:
df = df.reset_index()
df_raw = df.copy()

In [ ]:
# 클러스터링 수행
cluster_list = dm.cluster_mols(df.mol, 0.2)

## 클러스터링 결과 보기

In [ ]:
cluster_list

(((971, 966, 967, 968, 969, 970, 972, 973, 976, 977),
  (2022, 2023, 2024, 2025, 2044, 2046, 2047, 2052, 2054),
  (1063, 1044, 1051, 1052, 1059, 1064, 1066, 1068),
  (1061, 1042, 1048, 1060),
  (3357, 3355, 3356, 3358, 3359, 3362, 3363),
  (2826, 2819, 2820, 2821, 2827, 2828),
  (2050, 2051, 2060),
  (442, 414, 418, 438, 439, 440),
  (3466, 3472, 3473, 3474, 3475),
  (3346, 2177, 2178, 3340, 3345),
  (3330, 3121, 3122, 3331, 3332),
  (3320, 3311, 3314, 3317, 3322),
  (2936, 2932, 2933, 2934, 2935),
  (2867, 2864, 2868, 2870, 2872),
  (2492, 2490, 2493, 2494, 2495),
  (1799, 1796, 1797, 1798, 1800),
  (3279, 3280, 3281, 3282),
  (3196, 3174, 3188, 3194),
  (3151, 3143, 3146, 3150),
  (3066, 2926, 3063, 3065),
  (2871, 2860, 2861, 2862),
  (2566, 2559, 2564, 2565),
  (2056, 2032, 2033, 2034),
  (2039, 2040, 2041, 2057),
  (1939, 1936, 1937, 1938),
  (1680, 1151, 1155, 1676),
  (1633, 768, 769, 782),
  (1582, 1432, 1580, 1581),
  (1563, 1556, 1557, 1562),
  (1561, 1554, 1555, 1560),
  (10

In [ ]:
print(len(cluster_list)) # 각 클러스터들의 인덱스를 담은 튜플과, mol 객체를 담을 튜프 두 세트를 제공
print(len(cluster_list[0])) # 클러스터의 수
print(len(cluster_list[0][0])) # 첫번째 클러스트에 속한 샘플 수

2
3025
10


In [ ]:
cluster_list[0][0], cluster_list[1][0], len(cluster_list[0][0])

((971, 966, 967, 968, 969, 970, 972, 973, 976, 977),
 (<rdkit.Chem.rdchem.Mol at 0x7feb280c0ba0>,
  <rdkit.Chem.rdchem.Mol at 0x7feb280c0e40>),
 10)

In [ ]:
# 각 샘플별로 클러스터 번호를 배정
cluster_idx = [-1] * len(df)
for i,cluster in enumerate(tqdm(cluster_list[0])):
    # auto_align_many는 유사한 스캐폴드를 묶어주는 기능 제공
    # dm.align.auto_align_many([df.mol.values[x] for x in cluster],copy=False,partition_method='scaffold')

    # 같은 클러스터 내의 샘플에게 같은 클러스터 번호를 배정
    for c in cluster:
        cluster_idx[c] = i

# 샘플별 클러스터 번호를 데이터프레임 컬럼에 추가
df['cluster'] = cluster_idx

100%|██████████| 3025/3025 [00:00<00:00, 747482.60it/s]


- 각 클러스터별로 하나의 대표 샘플만 추출한 데이터프레임 생성 (중복되는 클러스터 번호의 샘플 삭제)

In [ ]:
cluster_sample_df = df.sort_values("cluster").drop_duplicates("cluster").copy()
cluster_sample_df.shape

(3025, 11)

- 각 클러스터의 크기를 value_counts를 이용하여 구하고 **cluster_sample_df**와 합친다(merge)

In [ ]:
cluster_count_df = df.cluster.value_counts().to_frame().reset_index()
cluster_count_df.columns = ["cluster","count"]
cluster_sample_df = cluster_sample_df.merge(cluster_count_df,on="cluster")
print(cluster_sample_df.shape)
cluster_sample_df[:3]

(3025, 12)


,index,SMILES,Name,pIC50,mol,mw,logp,hbd,hba,max_ring_size,cluster,count
0,1122,CC(C)Oc1ccc(cc1)C2CN(C(=O)N2CCc3ccc(cc3)OC)NS(...,CHEMBL571952,4.770,<rdkit.Chem.rdchem.Mol object at 0x7feb280c0c10>,447.183,2.968,1,8,6,0,10
1,2364,c1ccc(c(c1)C(=O)Nc2ccc(cc2)c3nnc(o3)NCCCCN4CCC...,CHEMBL2022733,4.900,<rdkit.Chem.rdchem.Mol object at 0x7feb280ed430>,437.223,4.806,2,7,6,1,9
2,1228,Cc1c(ocn1)c2nnc(n2C)SCCCN3CC4CC4(C3)c5ccc(cc5)...,CHEMBL1079823,6.400,<rdkit.Chem.rdchem.Mol object at 0x7feb280c4b30>,451.241,4.832,0,6,6,2,8


## 클러스터별 대표 샘플 보기
- [mols2grid](https://github.com/cbouy/mols2grid)를 사용하여 클러스터별 샘플을 볼 수 있다

In [ ]:
mols2grid.display(cluster_sample_df,mol_col="mol",subset=["img","cluster","count"])
# mols2grid.display(cluster_sample_df,mol_col="mol",subset=["img","cluster","count"],use_coords=True, prerender=True)

MolGridWidget()

## 동일 클러스터 내 샘플 보기

- **show_cluster** 함수를 정의
- 조건에 맞는 샘플을 리턴하는 query 함수를 사용

In [ ]:
def show_cluster(x):
    return mols2grid.display(df.query(f"cluster == {x}"),mol_col="mol",subset=["img","Name","pIC50"],
            use_coords=True, prerender=True, transform = {"pIC50" : lambda val: f"{val:.2f}"})

In [ ]:
show_cluster(2)

MolGridWidget()